# Tutorial 03: Simple Pendulum

The classic pendulum - where physics gets nonlinear! BE SURE to have mechanicsdsl installed BEFORE or combine with import code block via !pip install mechanicsdsl-core

## Physics
- Mass $m$ on rod of length $L$
- Lagrangian: $L = \frac{1}{2}mL^2\dot{\theta}^2 + mgL\cos\theta$
- Equation of motion: $\ddot{\theta} = -\frac{g}{L}\sin\theta$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mechanics_dsl import PhysicsCompiler

compiler = PhysicsCompiler()

In [ ]:
dsl_code = r"""
\system{simple_pendulum}

\defvar{theta}{Angle from vertical}{rad}

\parameter{m}{1.0}{kg}
\parameter{L}{1.0}{m}
\parameter{g}{9.81}{m/s^2}

\lagrangian{\frac{1}{2} * m * L^2 * \dot{theta}^2 + m * g * L * \cos{theta}}

\initial{theta=0.5, theta_dot=0.0}
"""

result = compiler.compile_dsl(dsl_code)
print("✓ Compiled!" if result['success'] else f"✗ {result['error']}")

In [ ]:
# Simulate
solution = compiler.simulate(t_span=(0, 10), num_points=1000)
t = solution['t']
theta = solution['y'][0]
theta_dot = solution['y'][1]

# Convert to Cartesian
L = 1.0
x = L * np.sin(theta)
y = -L * np.cos(theta)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Angle vs time
axes[0,0].plot(t, np.degrees(theta), 'b-', lw=2)
axes[0,0].set_xlabel('Time (s)')
axes[0,0].set_ylabel('Angle (degrees)')
axes[0,0].set_title('Pendulum Angle')
axes[0,0].grid(True, alpha=0.3)

# Phase portrait
axes[0,1].plot(theta, theta_dot, 'purple', lw=2)
axes[0,1].set_xlabel('θ (rad)')
axes[0,1].set_ylabel('θ̇ (rad/s)')
axes[0,1].set_title('Phase Portrait')
axes[0,1].grid(True, alpha=0.3)

# Trajectory
axes[1,0].plot(x, y, 'g-', lw=2)
axes[1,0].plot(0, 0, 'ko', ms=8)  # pivot
axes[1,0].set_xlabel('x (m)')
axes[1,0].set_ylabel('y (m)')
axes[1,0].set_title('Pendulum Trajectory')
axes[1,0].axis('equal')
axes[1,0].grid(True, alpha=0.3)

# Snapshot
axes[1,1].set_xlim(-1.5, 1.5)
axes[1,1].set_ylim(-1.5, 0.5)
axes[1,1].plot([0, x[-1]], [0, y[-1]], 'b-', lw=3)
axes[1,1].plot(0, 0, 'ko', ms=10)
axes[1,1].plot(x[-1], y[-1], 'ro', ms=15)
axes[1,1].set_title('Final Position')
axes[1,1].axis('equal')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Small Angle Approximation

For small angles ($\theta \ll 1$): $\sin\theta \approx \theta$

This gives simple harmonic motion with period $T = 2\pi\sqrt{L/g}$

In [ ]:
T_approx = 2 * np.pi * np.sqrt(1.0 / 9.81)
print(f"Approximate period (small angle): T ≈ {T_approx:.3f} s")

# Measure actual period from simulation
zero_crossings = np.where(np.diff(np.sign(theta)))[0]
if len(zero_crossings) >= 2:
    T_actual = 2 * (t[zero_crossings[1]] - t[zero_crossings[0]])
    print(f"Actual period (from simulation): T = {T_actual:.3f} s")